<a href="https://colab.research.google.com/github/tigerofmurder/bioinformatica/blob/main/blast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Bio

In [34]:
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
from difflib import SequenceMatcher


Datset_fasta = []
sequences = SeqIO.parse("/content/seqs2.fasta", "fasta")
for record in sequences:
  Datset_fasta.append(record.seq)

In [35]:
from Bio.Align import substitution_matrices
BLOSUM62 = substitution_matrices.load("BLOSUM62")
#20^3 no 23^3
headers_blosum62 = np.array(['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V'])
#print(BLOSUM62)


In [17]:
LETRA = "GAGTT"

In [12]:
def division(letter,k = 3):
  array = []
  for i in range(len(letter)-k):
    str1 = letter[i:i+k]
    array.append(str1)
  return array

In [7]:
import itertools
def get_combination(blosum, k=3):
  return [p for p in itertools.product(headers_blosum62, repeat=3)]

In [29]:
a = division(LETRA)
all_combinations = get_combination(headers_blosum62)

In [51]:
def find_neighbors(seeds, possibilities, blosum, threshold=13):
  local_neighbors = []
  index_seed = 0
  for i in seeds:
    for j in possibilities:
      score = 0
      for k in range(len(j)):
        score += blosum[i[k],j[k]] 
      if score>=threshold:
        local_neighbors.append((index_seed,''.join(j)))
    index_seed += 1
  return local_neighbors

In [52]:
neighbors = find_neighbors(a,all_combinations,BLOSUM62)
neighbors

[(0, 'GAG'), (0, 'GSG'), (1, 'AGT')]

In [73]:
def extend(palabra,  query, matrix, pos_palabra, pos_query, umbral=22):
  p_izq_q, p_der_q = pos_query[0],pos_query[1]
  p_izq_w, p_der_w = pos_palabra[0],pos_palabra[1]
  
  value = 0
  p_temp, q_temp = palabra[p_izq_w:p_der_w+1], query[p_izq_q:p_der_q+1]
  for i in range(len(p_temp)):
    value += matrix[p_temp[i], q_temp[i]]
  
  if value<umbral:
    return (None, None, -1)
  
  while p_izq_q >= 1 and p_izq_w >= 1 and p_der_w+1 < len(palabra) and p_der_q+1 < len(query):
    p_izq_q-=1
    p_izq_w-=1
    p_der_w+=1
    p_der_q+=1

    value = 0
    p_temp, q_temp = palabra[p_izq_w:p_der_w+1], query[p_izq_q:p_der_q+1]
    for i in range(len(p_temp)):
      value += matrix[p_temp[i], q_temp[i]]


    if value < umbral:
      p_der_w-=1
      p_der_q-=1
      p_izq_w+=1
      p_izq_q+=1
      break 

  value = 0
  p_temp, q_temp = palabra[p_izq_w:p_der_w+1], query[p_izq_q:p_der_q+1]
  for i in range(len(p_temp)):
    value += matrix[p_temp[i], q_temp[i]]

  return (palabra, palabra[p_izq_w:p_der_w+1], value)

In [76]:
def search_database(neighbors, DB, matrix, query, umbral = 22):
  salida = []
  for (index_seed, neighbor) in neighbors:
    for cadena in DB:
      #print(cadena)
      #print(neighbor)
      position_find = 0
      position_initial = 0
      while True:
        position_find = cadena.find(neighbor, position_initial)
        if position_find == -1:
          break
        valores = extend(cadena, query, matrix, (position_find, position_find+len(neighbor)-1), (index_seed, index_seed+len(neighbor)-1), umbral)
        if valores[2]!=-1:          
          salida.append(valores) 
        position_initial = position_find + 1
  return salida

In [77]:
search_database(neighbors,Datset_fasta,BLOSUM62,LETRA,11)

[(Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACAC...TTA'),
  Seq('GAG'),
  16.0),
 (Seq('AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACA